In [1]:
import math
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 2500)

# Loading historical data
hs_param = pd.read_csv('historical parameters.csv')
# Setting up the parameters differently
decay_factor = hs_param.decay_factor.iloc[0]
holding_period = hs_param.holding_period.iloc[0]
# print(holding_period)
confidence_level = hs_param.confidence_level.iloc[0]

rates_E = pd.read_csv('rates_instrument.csv')
rates_E = rates_E[rates_E["instrument_type"] == 'E']
rates_E.instrument = rates_E.instrument.astype('str')
rates_E["key"] = "instru" + '_'+rates_E.instrument

# Simulated Instrument rates
simulated_instruments = pd.DataFrame([])


In [2]:
rates_E

,date,instrument,instrument_type,value,key
0,20-Jul-22,Reliance,E,2503.00,instru_Reliance
1,19-Jul-22,Reliance,E,2437.10,instru_Reliance
2,13-Jul-22,Reliance,E,2422.25,instru_Reliance
3,12-Jul-22,Reliance,E,2401.80,instru_Reliance
4,06-Jul-22,Reliance,E,2397.15,instru_Reliance
5,05-Jul-22,Reliance,E,2377.55,instru_Reliance
6,29-Jun-22,Reliance,E,2420.45,instru_Reliance
7,28-Jun-22,Reliance,E,2423.90,instru_Reliance
8,22-Jun-22,Reliance,E,2391.40,instru_Reliance
9,21-Jun-22,Reliance,E,2388.10,instru_Reliance


In [3]:
rates_E.shape

(1255, 5)

In [4]:
for a, h in rates_E.groupby('instrument'):
    g = h.filter(["instrument", "instrument_type", "value", "key"])
    new_row = g.iloc[0].to_dict()
    new_row['simulated_rates'] = [0]
    top_row = pd.DataFrame(new_row)
    g = g.reset_index(drop = True)
    g['simulated_rates'] = np.log(g["value"]/g["value"].shift(-holding_period, fill_value = g["value"][0]))
    
    g = pd.concat([top_row, g]).reset_index(drop = True)

    g['new_value'] = g['simulated_rates'].apply(lambda x: np.exp(x)*g["value"].iloc[0])
    simulated_instruments = simulated_instruments.append(g)


/tmp/ipykernel_298/1023512331.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_instruments = simulated_instruments.append(g)
/tmp/ipykernel_298/1023512331.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_instruments = simulated_instruments.append(g)
/tmp/ipykernel_298/1023512331.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_instruments = simulated_instruments.append(g)
/tmp/ipykernel_298/1023512331.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_instruments = simulated_instruments.append(g)
/tmp/ipykernel_298/1023512331.py:12: FutureWarning: The frame.append method is deprecate

In [5]:
simulated_instruments

,instrument,instrument_type,value,key,simulated_rates,new_value
0,HDFCbank,E,1365.05,instru_HDFCbank,0.000000,1365.050000
1,HDFCbank,E,1365.05,instru_HDFCbank,-0.004532,1358.878033
2,HDFCbank,E,1348.05,instru_HDFCbank,-0.003370,1360.458120
3,HDFCbank,E,1347.55,instru_HDFCbank,-0.005993,1356.893835
4,HDFCbank,E,1362.05,instru_HDFCbank,0.006112,1373.419282
5,HDFCbank,E,1351.05,instru_HDFCbank,0.002260,1368.138578
6,HDFCbank,E,1357.85,instru_HDFCbank,0.010290,1379.168230
7,HDFCbank,E,1391.80,instru_HDFCbank,0.033535,1411.603083
8,HDFCbank,E,1406.15,instru_HDFCbank,0.036132,1415.273775
9,HDFCbank,E,1397.10,instru_HDFCbank,0.031483,1408.709821


In [9]:
simulated_instruments.shape

(1260, 6)

In [6]:
rates_exchange = pd.read_csv('rates_exchange.csv')

In [7]:
if len(rates_exchange) != 0:
    rates_exchange["key"] = rates_exchange["currency_from"] + "_"+rates_exchange["currency_to"]

simulated_exchange = pd.DataFrame([])
for i,k in rates_exchange.groupby('key'):
    j = k.filter(["currency_from", "currency_to", "rate", "key"])
    new_row = j.iloc[0].to_dict()
    new_row['simulated_rates'] = [0]
    top_row = pd.DataFrame(new_row)
    
    j = j.reset_index(drop = True)
    j['simulated_rates'] = np.log(j['rate']/j['rate'].shift(-holding_period, fill_value = j["rate"][0]))
    j = pd.concat([top_row, j]).reset_index(drop = True)
    j["new_rate"] = j['simulated_rates'].apply(lambda x: np.exp(x)*j["rate"].iloc[0])
    simulated_exchange = simulated_exchange.append(j)

simulated_exchange.head(5)

/tmp/ipykernel_298/37893595.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  simulated_exchange = simulated_exchange.append(j)


,currency_from,currency_to,rate,key,simulated_rates,new_rate
0,INR,INR,1,INR_INR,0.0,1.0
1,INR,INR,1,INR_INR,0.0,1.0
2,INR,INR,1,INR_INR,0.0,1.0
3,INR,INR,1,INR_INR,0.0,1.0
4,INR,INR,1,INR_INR,0.0,1.0


In [9]:
simulated_exchange.shape

(252, 6)

In [13]:
tx_eq

,id,instrument,instrument_id,currency,currency_base,unit_count,preferred_index,accounting_book,accounting_portfolio
0,Reliance1908,Reliance,16854,INR,INR,6000,NIFTY,1,1
1,TCS108,TCS,16876,INR,INR,10000,NIFTY,1,1
2,Infosys203,Infosys,16857,INR,INR,13550,NIFTY,1,1
3,Hindustanlever914,HLL,16858,INR,INR,7500,NIFTY,1,1
4,HDFCbank1942,HDFCbank,16859,INR,INR,11000,NIFTY,1,1


In [10]:

tx_eq = pd.read_csv('transaction_equity.csv')
n=hs_param['no_of_days'].iloc[0]

def get_mtm_bc(idx,tx_df, ex_df, instr_df):
    
    res_df = list()
    for k, h in tx_eq.groupby('id'):
        h["keyMatch"] = h['currency'] + '_' + h['currency_base']
        keyMatch = h["keyMatch"].iloc[0]
        instrMatch = h["instrument"].iloc[0]
        unitCount = h['unit_count'].iloc[0]
        
        crss_rate_df = ex_df[ex_df['key']==keyMatch]
        cross_rate = crss_rate_df['new_rate'].iloc[0]
        
        equity_rate_df = instr_df[instr_df['instrument'] == instrMatch]
        equity_rate = equity_rate_df['new_value'].iloc[0]
        
        mtm_bc = cross_rate * equity_rate * unitCount
        result = {'idx': idx, "env":k,"instrument": instrMatch, "unit_count":unitCount, "accounting_book":h["accounting_book"].iloc[0],
                 "accounting_portfolio": h["accounting_portfolio"].iloc[0], "currency":h["currency"].iloc[0], "mtm_bc":mtm_bc}
        res_df.append(result)

    return res_df



In [11]:
final_df = pd.DataFrame()
for i in range(0, n):
    exch_rates = simulated_exchange[simulated_exchange.index == i]
    instr_rates = simulated_instruments[simulated_instruments.index == i]
    result = get_mtm_bc(i, tx_eq, exch_rates, instr_rates)
    res = pd.DataFrame(result)

    final_df = final_df.append(res)

/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_

/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_

/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_

/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(res)
/tmp/ipykernel_298/2821243799.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_

In [12]:
final_df

,idx,env,instrument,unit_count,accounting_book,accounting_portfolio,currency,mtm_bc
0,0,HDFCbank1942,HDFCbank,11000,1,1,INR,1.501555e+07
1,0,Hindustanlever914,HLL,7500,1,1,INR,1.953375e+07
2,0,Infosys203,Infosys,13550,1,1,INR,2.053232e+07
3,0,Reliance1908,Reliance,6000,1,1,INR,1.501800e+07
4,0,TCS108,TCS,10000,1,1,INR,3.164800e+07
0,1,HDFCbank1942,HDFCbank,11000,1,1,INR,1.494766e+07
1,1,Hindustanlever914,HLL,7500,1,1,INR,2.036329e+07
2,1,Infosys203,Infosys,13550,1,1,INR,2.085785e+07
3,1,Reliance1908,Reliance,6000,1,1,INR,1.558557e+07
4,1,TCS108,TCS,10000,1,1,INR,3.071673e+07
